In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import  MinMaxScaler
import numpy as np # linear algebra
import pandas as pd
from yellowbrick.cluster import SilhouetteVisualizer
import seaborn as sns
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data= pd.read_csv('/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv')

In [ ]:
data.head()

In [ ]:
data.describe().T

In [ ]:
# Visualization of data
st=MinMaxScaler()

pc=PCA(n_components=9)
data_pca=pc.fit_transform(st.fit_transform(data.iloc[:,1:]))
# evaluating two clusters
sns.scatterplot(x=data_pca[:,0],y=data_pca[:,1])


In [ ]:
sum(pc.explained_variance_ratio_[0:2])

In [ ]:
sns.lineplot(x=np.arange(1,10),y=pc.explained_variance_ratio_*100)

In [ ]:
def train_KMeans(data,cluster):
    cluster_id,losses,silhoutte,kmean=[],[],[],[]
    st=MinMaxScaler()
    dt=st.fit_transform(data.iloc[:,1:].values)
    # Randomly training 10 models for a cluster to overcome poor random initialization 
    for i in range(10):
        km=KMeans(n_clusters=cluster,init='random',precompute_distances=True,
                  random_state=np.random.randint(0,100),n_jobs=-1,)
        pred=km.fit_predict(dt)
        cluster_id.append(pred)
        losses.append(km.inertia_)
        kmean.append(km)
 
        silhoutte.append(silhouette_score(dt,pred))
    return cluster_id,losses,silhoutte,kmean

# Elbow method

In [ ]:

loss=[]
labels=[]
silh=[]
km=[]
for i in [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]:
    cluster_id,losses,silhoutte,kmean=train_KMeans(data,i)
    loss.append(np.min(losses))
    km.append(kmean[np.argmax(silhoutte)])
    labels.append(cluster_id[np.argmin(losses)])
    silh.append(silhoutte)

In [ ]:
sns.lineplot(x=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],y=loss);
plt.title('SSE vs clusters ');
# The elbow method dosent give a proper elbow but we can estimate 6-8 to be the better option 

# Silhouette Analysis

In [ ]:
sns.lineplot(x=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],y=np.mean(np.array(silh),axis=1));
plt.title('silhouette_score vs clusters ');

In [ ]:
st=MinMaxScaler()
dt=st.fit_transform(data.iloc[:,1:].values)
pc=PCA(n_components=2)
df_2=pc.fit_transform(dt)
plt.figure(figsize=(15,15))
j=1
for i in range(4):
    plt.subplot(4,2,j)
    vis=SilhouetteVisualizer(km[i],colors = sns.color_palette('hls')[0:i+2],);
    vis.fit(dt);
    j+=1
    plt.subplot(4,2,j)
    sns.scatterplot(x=df_2[:,0],y=df_2[:,1],hue=labels[i],palette=sns.color_palette('hls')[0:i+2])
    j+=1

models 2,3 and 4 clusters shows good average Silhouette score and the individual clusters have a score above average value. Next we will look at countries belonging to these clusters and try to draw some inference. cluster 1 ofk =3 and k=4 are same

In [ ]:
pd.read_csv('../input/unsupervised-learning-on-country-data/data-dictionary.csv').style

In [ ]:
for i in range(3):
    data['cluster']=labels[i]
    print(data.groupby(['cluster']).mean().T)
       

##### for k=2, 


Cluster 0 countries have a high child mortality rate of 87, compared to an average of 15 in the rest of the world. Cluster 0 countries also have much lower life expectancy ,income and GDP. These countries also have a tendency to import more and export less than cluster 1 countries.This means that these are the countries with the least developed industrial and manufacturing sectors, resulting in lower employment, increased poverty, and higher living costs.

##### for k=3 
cluster 1 tends to be the least developed with child mortality of 93(this is much higher than that of cluster 0 in k=2), also lower gdp,income and life expectancy. 

##### for k=4 
cluster 1 tends to be the least developed but countries belongingto this cluster isexactly same asthat in cluster1 ofmodelwith k=3


In [ ]:
# arranging categories with similar scales
a=[['child_mort','inflation', 'health','total_fer',], ['exports','imports',  'life_expec'],['income',  'gdpp']]

for i in range(3):
    data['cluster']=labels[i]
    print(f'\n----------------k={i+2}-----------\n')  
    for j in a:
       
        data.groupby(['cluster']).mean()[j].T.plot(kind='bar')
        plt.show()
        
          
    

###### The CEO of the company should be focusing on countries belonging to cluster 1 of model with K=3

In [ ]:
data['cluster']=labels[1]
data[data['cluster']==1][['country']]